In [106]:
#Michael Timbes
#Purpose:
#Image_Classification Based on Logistic classification model. Single hidden layer, does not use convolutional layers
#does not use pooling.

In [107]:
import tensorflow as tf
import numpy as np
#Visualization
import matplotlib.pyplot as plt
#File import
from os import listdir
from os import path as opath
from PIL import Image as PImage

In [108]:
#Path of test images
path = ('train_set')
#Dimension of image
IMAG_X = 10

#Trainig rate alpha
alpha = 0.5

#Number of inputs defined
NUM_IN = (IMAG_X**2)

#Number of classifications
y_out_clss = 2

#Size of batch
BatchSize = 2

In [109]:
#Import data
def ImportImages(path):
    """
    def ImportImages(path):
    USES VARIABLES: IMAG_X, IMAG_Y, expects these to be set already.
    If prefered, edit code to pass them as arguments.
    ________________________________________________________________
    Function Outline:
    1. Loads list of images from the path variable (type string).
    2. Iterates through directory loads image into I/O file stream.
    3. Converts file to LA grey scale and resizes.
    4. Casts the image data into Numpy array.
    5. Returns Numpy array object.
    """

    imagesList = listdir(path)
    
    loadedImages = []
    
    loadedLabels = []
  
    new_size = IMAG_X, IMAG_X
    
    for image in imagesList:
     
        img = PImage.open(path +'/'+ image)
        #Pull file name from current image- use it as a label
        loadedLabels.append(opath.splitext(image)[0])
        img.load()
        #Resize step- ensures that all images follow.
        img.thumbnail(new_size, PImage.ANTIALIAS )
        loadedImages = np.asarray( img, dtype="int32" )
    return loadedImages, loadedLabels

In [110]:
def shape_up_X(train_X, IMAG_X):
    """
    shape_up_X(train_X, IMAG_X):
    
    Expects a 3D numpy array train_X with 
    dimensions (width_val, height_val, image#).
    Must be square matrix with and height being
    equal.
    Returns new_X which is a reshaped train_X
    of type numpy array.
    ____________________________________
    Dimensions are (pixels, image#). The
    size of the pixels dimension is taken from
    the square of IMAG_X. The number of images are found
    by taking the length of the third column.
    """
    num_exs = len(train_X[0,0,:])
    new_X = train_X.reshape((IMAG_X**2, num_exs ))
    new_X.flatten('C')
    return new_X

In [111]:
def create_batch(train_X, train_Y, start, batch_size):
    """
    def create_batch(train_X, batch_size):
    __________________________________________
    Function Outline:
    1.Creates training subset of train_X and train_Y
    """
    batch_x = train_X[:,start:batch_size]
    batch_y = train_Y[start:batch_size]
    return batch_x, batch_y

In [112]:
train_X, train_Y = ImportImages(path)
Num_to_Train = len(train_X[0,0,:])
train_X = shape_up_X(train_X,IMAG_X)
Num_to_Train

3

In [113]:
#The x_input_layer and y_output_layer values are placeholders for the model that accept the flattened image (x) 
#and then the ouput of theclassifications (y). 
#
x_input_layer = tf.placeholder(tf.float32, shape= [None,NUM_IN])
y_output_layer = tf.placeholder(tf.float32, shape= [None, y_out_clss])


#The vectors for weights and b- the bias will be defined as variables for training later.

Weights = tf.Variable(tf.zeros([NUM_IN, y_out_clss]))
b = tf.Variable(tf.zeros([y_out_clss]))

In [114]:
#Outline of the model based on the probabilities calculated plus bias values.

y_model = tf.matmul(x_input_layer, Weights) + b

In [115]:
#This is where the train steps happens. Cross entropy is calculated by running the current model and then running 
#gradient decent. Training step stores results from the gradient descent minimizing cost function (cross_entropy).


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_output_layer, logits = y_model))

train_step = tf.train.GradientDescentOptimizer(alpha).minimize(cross_entropy)

In [116]:
#Init session and global variables
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

In [121]:
#Training block
for i in range(1): #Outside train loop
    end = BatchSize
    for start in range(0,Num_to_Train,BatchSize):
        end = start + BatchSize
        batch_x, batch_y = create_batch(train_X, train_Y, start, end)
        #print(batch_x[1], batch_y[1])
    train_step.run(feed_dict={x_input_layer: batch_x[:,start:end], y_output_layer: batch_y[start:end]}) 

ValueError: Cannot feed value of shape (100, 0) for Tensor 'Placeholder_12:0', which has shape '(?, 100)'